In [1]:
%load_ext tensorboard

In [2]:
# import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# read data
df = pd.read_csv("ML_data_unscaled_dummys_encoded.csv")
df = df.drop(['Entry_ID','BPM'],axis=1)
df.head()

,Age,Hours_per_day,Anxiety,Depression,Insomnia,OCD,While_working_No,While_working_Yes,Instrumentalist_No,Instrumentalist_Yes,...,Rock_Rarely,Rock_Sometimes,Rock_Very frequently,Video game music_Never,Video game music_Rarely,Video game music_Sometimes,Video game music_Very frequently,Music_effects_Improve,Music_effects_No effect,Music_effects_Worsen
0,18.0,4.0,7.0,7.0,10.0,2.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,61.0,2.5,9.0,7.0,3.0,3.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,18.0,4.0,7.0,2.0,5.0,9.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,18.0,5.0,8.0,8.0,7.0,7.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,18.0,3.0,4.0,8.0,6.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Depression model

In [4]:
# split data into features and target
y = df.Depression.values
X = df.drop('Depression', axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    # activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32, max_value=512, step=32), activation=hp.Choice('activation',['relu','tanh','sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
    
    nn_model.add(tf.keras.layers.Dense(units=1))

    # Compile the model
    nn_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [7]:
# Import the kerastuner library
import kerastuner as kt

directory = 'logs/dep_hyp'

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2,
    directory=directory,
    overwrite=True,
    project_name='dep_hyp')

c:\Users\mbtho\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [8]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f'{directory}/fit', histogram_freq=1)

In [9]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test),callbacks=[tensorboard_callback])

Trial 180 Complete [00h 00m 04s]
val_accuracy: 0.0234375

Best val_accuracy So Far: 0.140625
Total elapsed time: 00h 06m 29s
INFO:tensorflow:Oracle triggered exit


In [10]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'first_units': 224,
 'activation': 'tanh',
 'num_layers': 1,
 'units_0': 480,
 'units_1': 256,
 'units_2': 32,
 'units_3': 192,
 'units_4': 448,
 'units_5': 128,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [11]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 4.4850 - accuracy: 0.1406 - 130ms/epoch - 33ms/step
Loss: 4.484975814819336, Accuracy: 0.140625


In [11]:
#best_model.save("models/trained_dep.h5")

In [13]:
%tensorboard --logdir logs/dep_hyp